In [1]:
import smatch
import json

In [2]:
with open('wino_amrs.jsonl') as f:
    gt_amrs = [json.loads(l) for l in f]

In [3]:
with open('wino_blip2-flant5xl_caps_parse.jsonl') as f:
    cand_amrs = [json.loads(l) for l in f]

In [4]:
gt_amrs = gt_amrs[:len(cand_amrs)]

In [5]:
SAMPLE_SIZE=len(cand_amrs[0]['parses_0'])

In [10]:
help(smatch)

Help on module smatch:

NAME
    smatch

DESCRIPTION
    This script computes smatch score between two AMRs.
    For detailed description of smatch, see http://www.isi.edu/natural-language/amr/smatch-13.pdf

FUNCTIONS
    compute_f(match_num, test_num, gold_num)
        Compute the f-score based on the matching triple number,
                                     triple number of AMR set 1,
                                     triple number of AMR set 2
        Args:
            match_num: matching triple number
            test_num:  triple number of AMR 1 (test file)
            gold_num:  triple number of AMR 2 (gold file)
        Returns:
            precision: match_num/test_num
            recall: match_num/gold_num
            f_score: 2*precision*recall/(precision+recall)
    
    compute_match(mapping, weight_dict)
        Given a node mapping, compute match number based on weight_dict.
        Args:
        mappings: a list of node index in AMR 2. The ith element (value j) mea